<a href="https://colab.research.google.com/github/vrod237/Module_16_Big_Data_Analysis/blob/master/Module_16_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-2.4.7'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Fetched 252 kB in 2s (123 kB/s)
Reading package lists... Done


In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-09-21 01:31:26--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.05MB/s    in 0.8s    

2020-09-21 01:31:28 (1.05 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Sports_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

sport_review_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Sports_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

# Show DataFrame
sport_review_data_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...|2015-08-31 00:00:00|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...| 

In [ ]:
#Count the rows of the dataframe
sport_review_data_df.count()

4850360

In [ ]:
#Drop null values
dropna_df = sport_review_data_df.dropna()
dropna_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   48945260|R1WBPB8MDCCN8F|B012P7UPSM|     409940130|Chicago Blackhawk...|          Sports|          5|            0|          0|   N|                N|   LOVE IT. 6 stars!|Bought this last ...|2015-08-31 00:00:00|
|         US|    5782091|R32M0YEWV77XG8|B001GQ3VHG|     657746679|Copag Poker Size ...| 

In [ ]:
#Count the rows of the dataframe
dropna_df.count()

4848999

In [ ]:
#Load in a sql function to use columns
from pyspark.sql.functions import col

In [ ]:
# Drop Duplicates for the dataframe
cleaned_df = dropna_df.drop_duplicates()
cleaned_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   16250654|R1009D4P4FS0H5|B002WKDZIK|     432638981|MLB Head Rest Cov...|          Sports|          5|            0|          0|   N|                Y|           Great fit|I was surprised t...|2014-12-07 00:00:00|
|         US|    3194133|R100KRNEKW216J|B004XWKZTW|     857930697|RDX Cowhide Leath...| 

In [ ]:
#Count the rows of the dataframe
cleaned_df.count()

4848999

In [ ]:
from pyspark.sql.functions import to_date

date_df = cleaned_df.withColumn('review_date', 
                   to_date(cleaned_df.review_date, 'yyyy-MM-dd'))
date_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16250654|R1009D4P4FS0H5|B002WKDZIK|     432638981|MLB Head Rest Cov...|          Sports|          5|            0|          0|   N|                Y|           Great fit|I was surprised t...| 2014-12-07|
|         US|    3194133|R100KRNEKW216J|B004XWKZTW|     857930697|RDX Cowhide Leath...|          Sports|          5|    

In [ ]:
# Create review dataframe to match review_id table
clean_review_df = date_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
clean_review_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1009D4P4FS0H5|   16250654|B002WKDZIK|     432638981| 2014-12-07|
|R100KRNEKW216J|    3194133|B004XWKZTW|     857930697| 2015-08-11|
|R100RLLUBWF8LV|   37034064|B00IRLI45C|     830961924| 2015-04-07|
|R100SQRRYXEXRS|   13226850|B006GPXB6S|     241865321| 2014-07-16|
|R100XCP47QLYGY|    8353218|B00HWH214C|     439871673| 2015-04-11|
|R101COSNZ9CHZZ|   48270725|B003RCCZK6|     444216576| 2013-02-19|
|R101J2DDFFWXSA|    1557701|B00EEE2XPS|     836866547| 2014-11-16|
|R101RFMV849UM0|   48312776|B00HZJ8J4S|     896316279| 2015-07-10|
|R101WR8MV7RHVJ|   17884222|B00QJ22INK|     161738896| 2015-02-05|
|R1022R2XT4MODW|   16716265|B004TDPTZU|     190903720| 2013-01-14|
|R102CD34PITWAD|   42738332|B000LC7ZUU|     426085597| 2009-11-04|
|R102S8PN85DDL5|   46812936|B000NOOSII|     201962328| 2015-03

In [ ]:
# Create products dataframe to match products table
clean_products_df = date_df.select(["product_id", "product_title"]).drop_duplicates()
clean_products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00QKTNMDC|Jarv SMART BT Blu...|
|B002WU9PIE|Fighting Sports P...|
|B000P3WQ86|Plano Pro-Max Sin...|
|B0012K9RFQ|ak887 9mm silver ...|
|B000BJKWXK|CAP Barbell Olymp...|
|B00SIKOKS2|Prime 10 Year Bas...|
|B00ISHW3D4|Nike Mens Therma ...|
|B000YJ303G|Speedo Women's Pl...|
|B0063HVE18|Weston 30-0008-W ...|
|B002BK3ZSQ|MLB Anaheim Angel...|
|B006H7PKFU|United States Mar...|
|B003MSQ5AG|The Weather Compa...|
|B000VZLCWY|Lube-N-Walk Tread...|
|B006DY3R3Y|MAG SIG P250-SC 4...|
|B00DTVCFW8|adidas Performanc...|
|B00CJICDT2|TOUGH-GRID 750lb ...|
|B00G50R6QA|Hotdog Yoga Rollpack|
|B00HX9LIDE|Hooded Deluxe Rai...|
|B009TN5JK0|VERY100 Red Dot B...|
|B00FGUHKY8|Wacces Resistance...|
+----------+--------------------+
only showing top 20 rows



In [ ]:
# Create customers dataframe to match customers table

clean_customers_df = date_df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")
clean_customers_df.show(10)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    7325573|             6|
|   43272334|             2|
|   42103966|             1|
|   18845343|             3|
|   31444826|             7|
|    7434128|             1|
|   38058353|             1|
|   15367185|            10|
|   53036614|             2|
|   48264549|             9|
+-----------+--------------+
only showing top 10 rows



In [ ]:
# Create vine_table dataframe to match vine table
vine_table_df = cleaned_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_table_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1009D4P4FS0H5|          5|            0|          0|   N|
|R100KRNEKW216J|          5|            0|          0|   N|
|R100RLLUBWF8LV|          5|            0|          0|   N|
|R100SQRRYXEXRS|          4|            0|          1|   N|
|R100XCP47QLYGY|          5|            0|          0|   N|
|R101COSNZ9CHZZ|          4|            0|          0|   N|
|R101J2DDFFWXSA|          5|            0|          0|   N|
|R101RFMV849UM0|          1|            1|          1|   N|
|R101WR8MV7RHVJ|          5|            0|          0|   N|
|R1022R2XT4MODW|          4|            2|          2|   N|
|R102CD34PITWAD|          2|            0|          0|   N|
|R102S8PN85DDL5|          4|            0|          0|   N|
|R1038EUSHFX2OF|          5|            0|          0|   N|
|R103EQEESLK11F|          5|            

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz-challenge.ct8qvq3ky7mv.us-east-2.rds.amazonaws.com:5432/challenge_db"
config = {"user":"postgres", 
          "password":"Longhorn74!", 
          "driver":"org.postgresql.Driver"}


In [ ]:
# Write dataframe to vine table table in RDS
vine_table_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

KeyboardInterrupt: ignored

In [ ]:
# Write DataFrame to review_id_table table in RDS
clean_review_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write dataframe to products table in RDS
clean_products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write dataframe to customers table in RDS
clean_customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:


vine_table_df.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1009D4P4FS0H5|          5|            0|          0|   N|
|R100KRNEKW216J|          5|            0|          0|   N|
|R100RLLUBWF8LV|          5|            0|          0|   N|
|R100SQRRYXEXRS|          4|            0|          1|   N|
|R100XCP47QLYGY|          5|            0|          0|   N|
|R101COSNZ9CHZZ|          4|            0|          0|   N|
|R101J2DDFFWXSA|          5|            0|          0|   N|
|R101RFMV849UM0|          1|            1|          1|   N|
|R101WR8MV7RHVJ|          5|            0|          0|   N|
|R1022R2XT4MODW|          4|            2|          2|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [ ]:
# Filter the df with total votes more than 20
vine_df2 = vine_table_df.filter(vine_table_df["total_votes"] >= 20)
vine_df2.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| R4X5EY62I7HO1|          5|           46|         52|   N|
|R3AKWZ7O3FVVMS|          1|           22|         23|   N|
|R2KZSN9SFY3M59|          5|           23|         23|   N|
| R4RS735HKSPMW|          5|           53|         54|   N|
|R22UHTP1GC8G9O|          1|           73|         74|   N|
| RZVCN1W3IF1XL|          1|           41|         52|   N|
|R17HFK5K15M9PU|          5|           31|         32|   N|
|R33E4PLGZNK67W|          3|           19|         20|   N|
|R3AVB9RYTWCRMY|          5|           21|         24|   N|
|R1LEBX6TC665NJ|          5|           18|         20|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [ ]:
# Filter the above df to include the rows who have helpful votes more than 50%
vine_df3 = vine_df2.filter(vine_df2["helpful_votes"]/vine_df2["total_votes"] >= 0.5)
vine_df3.show(10)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| R4X5EY62I7HO1|          5|           46|         52|   N|
|R3AKWZ7O3FVVMS|          1|           22|         23|   N|
|R2KZSN9SFY3M59|          5|           23|         23|   N|
| R4RS735HKSPMW|          5|           53|         54|   N|
|R22UHTP1GC8G9O|          1|           73|         74|   N|
| RZVCN1W3IF1XL|          1|           41|         52|   N|
|R17HFK5K15M9PU|          5|           31|         32|   N|
|R33E4PLGZNK67W|          3|           19|         20|   N|
|R3AVB9RYTWCRMY|          5|           21|         24|   N|
|R1LEBX6TC665NJ|          5|           18|         20|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 10 rows



In [ ]:
# divide the above data into two categories votes which were a part of reward program with Vine as yes
yes_vine_df = vine_df3.filter(vine_df3["vine"] == "Y")

In [ ]:
# divide the above data into two categories votes which were a part of reward program with Vine as no
no_vine_df = vine_df3.filter(vine_df3["vine"] == "N")

In [ ]:
# show the statistics for reward program with Vine as yes
yes_vine_df.describe().show()

+-------+--------------+-----------------+-----------------+-----------------+----+
|summary|     review_id|      star_rating|    helpful_votes|      total_votes|vine|
+-------+--------------+-----------------+-----------------+-----------------+----+
|  count|           334|              334|              334|              334| 334|
|   mean|          null|4.098802395209581|65.01497005988024|72.36826347305389|null|
| stddev|          null|0.973738532463338|84.58385900781397|94.57132147550898|null|
|    min|R10V1GEWC8KK40|                1|               13|               20|   Y|
|    max| RZL21K50M52KW|                5|              720|              928|   Y|
+-------+--------------+-----------------+-----------------+-----------------+----+



In [ ]:
# show the statistics for reward program with Vine as no
no_vine_df.describe().show()

+-------+--------------+------------------+------------------+------------------+-----+
|summary|     review_id|       star_rating|     helpful_votes|       total_votes| vine|
+-------+--------------+------------------+------------------+------------------+-----+
|  count|         61589|             61589|             61589|             61589|61589|
|   mean|          null|3.8774943577586907|  50.9515173164039| 56.09922226371592| null|
| stddev|          null| 1.476267050453788|103.97497675828791|108.99226354759766| null|
|    min|R1002IDT34USH7|                 1|                10|                20|    N|
|    max| RZZTD14VO9O1A|                 5|              7379|              7512|    N|
+-------+--------------+------------------+------------------+------------------+-----+



In [ ]:
# total count for reward program with Vine as yes
yes_vine_count = yes_vine_df.count()
yes_vine_count

334

In [ ]:
# Count the data who have given five star rating for reward program with Vine as yes
yes_vine_five = yes_vine_df[yes_vine_df['star_rating']== 5].count()
yes_vine_five

139

In [ ]:
# percentage of five star rating on the total number of data for reward program with Vine as yes
percentage_five_vine_yes = (float(yes_vine_five) / float(yes_vine_count)) * 100
percentage_five_vine_yes

41.61676646706587

In [ ]:
# total count for reward program with Vine as no
no_vine_count = no_vine_df.count()
no_vine_count

61589

In [ ]:
# Count the data who have given five star rating for reward program with Vine as no
no_vine_five = no_vine_df[no_vine_df['star_rating']== 5].count()
no_vine_five

32660

In [ ]:
# percentage of five star rating on the total number of data for reward program with Vine as no
percentage_five_vine_no = (float(no_vine_five) / float(no_vine_count)) * 100
percentage_five_vine_no

53.028949974833175

In [ ]:
# percentage of five star rating with vine reward program as compared to without vine reward program 
five_star_biased_percent = (float(yes_vine_five) / float(no_vine_five)) * 1000
five_star_biased_percent

4.255970606246173

In [ ]:
Objective: Determine if Vine reviews are biased using appropriate statistical methods.

Data: In cleaning the data, we dropped and null and duplicate values. Then we only pulled reviews that had 
more than 20 total votes. Lastly, we pulled the reviews that had more than 50% of helpful votes.

Analysis: The data was divided into two categories: data where vine = "Y" which means that the voters were 
part of the reward program and data where vine = "n" which means that the voters were not apart of the 
reward program.

Conclusion: 

Where Vine = "Y", the total count is 334, the mean for star_rating was 4.09, helpful_votes 65.01 and 
total votes was 72.36. The five star rating given was 41.61%

Where Vine = "N", the total count is 61589, the mean for star_rating was 3.87, helpful_votes 50.95 and 
total votes was 56.09. The five star rating given was 53.02%

The total percentage of 5 star reviews of where vine = "Y" and "N", is only 4.25%. The reviews are not 
biased as a large portion of individuals not apart of the reward program still gave five start ratings 
and over half the reviews were listed a helpful.